In [1]:
## save variables
import pickle
## folder names
from glob import glob
## standard libraries
import numpy as np
## division for train and test
from sklearn.model_selection import train_test_split
##
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
#import sys
#!{sys.executable} -m pip install --upgrade https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-1.8.0-cp36-cp36m-linux_x86_64.whl --user
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D ,Convolution2D, MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.optimizers import SGD
from keras.utils import np_utils

/home/tlc/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
## Return a zero np.array with the only one in index position and with length given
def labelCreator (index, dim):
    try:
        a = np.zeros(dim, dtype = 'int')
        a [:,index] = 1
        return a
    except:
        return -1

def xyCreator(dic):
    X = []
    Y = []
    if type(dic)==dict:
        length = len(dic)
        for count, key in enumerate(dic):
            tmp = dic[key]
            label = labelCreator(count, [tmp.shape[0], length])
            X.append(tmp)
            Y.append(label)
    else:
        return -1
    return np.vstack(X) , np.vstack(Y)  

def xyCreatorCat(dic):
    X = []
    Y = []
    if type(dic)==dict:
        length = len(dic)
        for count, key in enumerate(dic):
            tmp = dic[key]
            label = np.array(count)
            label = np.resize(label, (tmp.shape[0],1))
            #label = labelCreator(count, [tmp.shape[0], length])
            X.append(tmp)
            Y.append(label)
    else:
        return -1
    return np.vstack(X) , np.vstack(Y)  

In [3]:
with open('variables/rawDict.pkl', 'rb') as f:  
    rawDict = pickle.load(f)
with open('variables/noiseDict.pkl', 'rb') as f:  
    noiseDict = pickle.load(f)    
with open('variables/mfccDict.pkl', 'rb') as f:  
    mfccDict = pickle.load(f)
with open('variables/fbankDict.pkl', 'rb') as f:  
    fbankDict = pickle.load(f)    
    
coreKey = ["Yes", "No", "Up", "Down", "Left", "Right", "On", "Off", "Stop", "Go", "Zero",
           "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine"]
#subdict = {k: bigdict[k] for k in bigdict.keys() & {'l', 'm', 'n'}}

In [4]:
X , Y = xyCreatorCat({k: mfccDict[k] for k in mfccDict.keys() & {'one', 'two', 'three'}})

X = normalize(X, axis=1)
pca = PCA(n_components=0.95)
print("Number of features before PCA:", X.shape[1])
X = pca.fit_transform(X)
print("Number of features after PCA at 95% of variance:", X.shape[1])

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

Number of features before PCA: 1287
Number of features after PCA at 95% of variance: 375
(5679, 375) (5679, 1)
(1420, 375) (1420, 1)


In [9]:
y_tr = np_utils.to_categorical(y_train, 3)
y_t = np_utils.to_categorical(y_test, 3)
X_train_r = np.reshape(X_train, ( X_train.shape[0],1, X_train.shape[1],1))
X_test_r = np.reshape(X_test, ( X_test.shape[0],1, X_test.shape[1],1))


In [15]:
cnn = Sequential()
cnn.add(Convolution2D(64, 4,  strides = 1,
    padding="same",
    activation="relu",
    input_shape=(1, X_train.shape[1], 1)))
#cnn.add(Convolution2D(64, 4, strides = 1, padding="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))

#cnn.add(Convolution2D(128, 3, strides = 1, padding="same", activation="relu"))
#cnn.add(Convolution2D(128, 3, strides =  1, padding="same", activation="relu"))
cnn.add(Convolution2D(128, 3,  strides = 1, padding="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))
   
#cnn.add(Convolution2D(256, 3, strides =  1, padding="same", activation="relu"))
#cnn.add(Convolution2D(256, 3,  strides = 1, padding="same", activation="relu"))
cnn.add(Convolution2D(256, 3,  strides = 1, padding="same", activation="relu"))
cnn.add(MaxPooling2D(pool_size=(2,1)))
   
cnn.add(Flatten())
cnn.add(Dense(128, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(y_tr.shape[1], activation="softmax"))

# define optimizer and objective, compile cnn

cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [ ]:
nb_epoch = 20
cnn.fit(X_train_r, y_tr, epochs=nb_epoch, validation_data=(X_test_r, y_t), batch_size=32)

Train on 5679 samples, validate on 1420 samples
Epoch 1/20
5679/5679 [==============================] - 32s 6ms/step - loss: 0.5614 - acc: 0.7260 - val_loss: 0.5379 - val_acc: 0.7592
Epoch 2/20
5679/5679 [==============================] - 32s 6ms/step - loss: 0.5214 - acc: 0.7598 - val_loss: 0.4833 - val_acc: 0.7951
Epoch 3/20
1440/5679 [======>.......................] - ETA: 20s - loss: 0.4462 - acc: 0.8125

In [18]:
preds = cnn.predict(X_test_r)

### END CODE HERE ###
print(preds)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

[[7.6144356e-03 4.2727000e-01 5.6511557e-01]
 [6.0666510e-04 4.7464275e-01 5.2475065e-01]
 [1.6914908e-04 2.9235905e-01 7.0747179e-01]
 ...
 [8.3474112e-01 1.2958294e-01 3.5675924e-02]
 [1.5717975e-03 4.6157557e-01 5.3685266e-01]
 [3.7446126e-04 4.1551203e-01 5.8411348e-01]]
Loss = [0.00761444 0.42727    0.5651156 ]
Test Accuracy = [0.00060667 0.47464275 0.52475065]
